In [6]:
!pip install streamlit
!pip install pdfplumber
!pip install ollama
!pip install langchain_community
!pip install langchain-ollama
!pip install unstructured
!ip install sentence_transformers
!pip install faiss-cpu
!pip install IPython
!pip install lightrag[ollama]


Object "install" is unknown, try "ip help".
  Using cached ollama-0.2.1-py3-none-any.whl.metadata (4.2 kB)
Using cached ollama-0.2.1-py3-none-any.whl (9.7 kB)
  Attempting uninstall: ollama
    Found existing installation: ollama 0.4.4
    Uninstalling ollama-0.4.4:
      Successfully uninstalled ollama-0.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-ollama 0.2.1 requires ollama<1,>=0.3.0, but you have ollama 0.2.1 which is incompatible.


In [7]:
import os
import threading
import subprocess
import time
import streamlit as st
import pdfplumber
from IPython.display import clear_output
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from IPython.display import display, Markdown

# Function to start Ollama API server
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
time.sleep(5)
from IPython.display import clear_output

!ollama pull llama3.1:8b
clear_output()

!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient


qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):

    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}

}
qa = SimpleQA(**model)
output=qa("what is happiness")
display(Markdown(f"**Answer:** {output.data}"))

from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()


# Load PDF
local_path = st.file_uploader("Upload a PDF file", type=["pdf"])
if local_path:
    with open("uploaded_file.pdf", "wb") as f:
        f.write(local_path.getbuffer())
    loader = UnstructuredPDFLoader(file_path="uploaded_file.pdf")
    data = loader.load()
    st.write(f"PDF loaded successfully: {local_path.name}")

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(data)
    st.write(f"Text split into {len(chunks)} chunks")

    # Create embeddings and vector database
    from langchain.embeddings import HuggingFaceEmbeddings
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    from langchain.vectorstores import FAISS
    vectordb = FAISS.from_documents(chunks, embeddings)
    st.write("Vector database created successfully")

    local_model = "llama3.2"
    llm = ChatOllama(model=local_model)

    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate 2
        different versions of the given user question to retrieve relevant documents from
        a vector database. By generating multiple perspectives on the user question, your
        goal is to help the user overcome some of the limitations of the distance-based
        similarity search. Provide these alternative questions separated by newlines.
        Original question: {question}""",
    )

    # Set up retriever
    retriever = MultiQueryRetriever.from_llm(
        vectordb.as_retriever(),
        llm,
        prompt=QUERY_PROMPT
    )

    template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # Function to chat with the PDF
    def chat_with_pdf(question):
        return chain.invoke(question)

    # Input question
    question = st.text_input("Ask a question about the PDF:")
    if question:
        st.write(Markdown(f"**Answer:** {chat_with_pdf(question)}"))

else:
    st.write("Upload a PDF file")



2024-12-17 07:07:01.152 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 07:07:01.156 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 07:07:01.158 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 07:07:01.171 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 07:07:01.350 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-17 07:07:01.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-17 07:07:01.357 Thread 'MainThread'

In [8]:
!pip install streamlit
!pip install pi_heif
!pip install
!pip install pdfplumber
!pip install ollama
!pip install langchain_community
!pip install langchain-ollama
!pip install unstructured
!pip install chromadb
!pip install sentence_transformers
!pip install faiss-cpu
!pip install IPython
!pip install lightrag[ollama]
!apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh


ERROR: You must give at least one requirement to install (see "pip help install")
  Using cached ollama-0.4.4-py3-none-any.whl.metadata (4.7 kB)
Using cached ollama-0.4.4-py3-none-any.whl (13 kB)
  Attempting uninstall: ollama
    Found existing installation: ollama 0.2.1
    Uninstalling ollama-0.2.1:
      Successfully uninstalled ollama-0.2.1
  Using cached ollama-0.2.1-py3-none-any.whl.metadata (4.2 kB)
Using cached ollama-0.2.1-py3-none-any.whl (9.7 kB)
  Attempting uninstall: ollama
    Found existing installation: ollama 0.4.4
    Uninstalling ollama-0.4.4:
      Successfully uninstalled ollama-0.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-ollama 0.2.1 requires ollama<1,>=0.3.0, but you have ollama 0.2.1 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is al

In [9]:

!pip install --upgrade langchain


In [12]:
# Install necessary packages
!pip install --upgrade ollama
!pip install streamlit
!npm install -g localtunnel

# Import the threading module
import threading

# Start the Streamlit app
def run_streamlit():
    subprocess.Popen(['streamlit', 'run', '/content/streamlit_app.py'])

thread = threading.Thread(target=run_streamlit)
thread.start()

# Expose the Streamlit app using localtunnel
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 940ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧⠙⠹⠸⠼⠴your url is: https://wicked-pillows-talk.loca.lt
^C


In [11]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.86.22.170